# Financial Analysis of Top German Companies

In this notebook, we load and analyze key financial metrics for several major German companies.
We perform data transformations, compute statistical measures, group by business sector, and create various plots to visualize trends in
revenue, net income, return on assets (roa), and return on equity (roe)

In [16]:
%useLatestDescriptors
%use dataframe@kc25, kandy@kc25

In [17]:
// Read data from a CSV file into a DataFrame
val dataFrame = DataFrame.read("top_12_german_companies.csv")
    .renameToCamelCase()

In [18]:
dataFrame.head()

company,period,revenue,netIncome,liabilities,assets,equity,roa,roe,debtToEquity,percentageDebtToEquity
Volkswagen AG,12/31/2017,9750496618,516889818.400000,21354201295,54861302788,33507101493,942.175.618,1.542.627.668,637.303.746,"0,00%"
Siemens AG,12/31/2017,19716237464,1276840007.000000,45009303223,75268101508,30258798286,1.696.389.282,4.219.731.382,1.487.478.214,"283,68%"
Allianz SE,12/31/2017,19458831198,1600107100.000000,48538978480,69583711255,21044732775,2.299.542.624,7.603.361.452,2.306.466.848,"329,65%"
BMW AG,12/31/2017,18808147150,960184349.600000,35382107627,67327482638,31945375011,142.614.028,3.005.706.927,1.107.581.539,"0,00%"
BASF SE,12/31/2017,16895580815,1797081911.000000,28309420014,68036567115,39727147101,2.641.347.127,4.523.561.449,71.259.635,"634,80%"


## Data Preparation: Formatting and Categorization

In this step, we prepare and clean the data for analysis.

- Custom Date Format: We define a custom date format (MM/DD/YYYY) to parse the "period" column into `LocalDate` without zero-padding for months.
- Business Sectors: We create an `enum` to classify companies into sectors such as Automotive, Banking, IT, and others.
- Data Transformation:
    - Convert the "period" column to `LocalDate` using the custom format.
    - Parse the "percentageDebtToEquity" column by removing the percentage sign and converting it to a `Double`.
    - Sort the data by "company" and "period".
    - Add a new column, "sector," which assigns companies to specific business sectors based on their names.


This step ensures the dataset is well-structured and categorized for further analysis.

In [19]:
import kotlinx.datetime.format.Padding
import kotlinx.datetime.format.char

// Define a custom date format without zero-padding for the month,
// separating month/day/year with slashes
val format = LocalDate.Format {
    monthNumber(Padding.NONE)
    char('/')
    dayOfMonth()
    char('/')
    year()
}

// Enum of Business Sectors
enum class BusinessSector(val simpleName: String) {
    AUTOMOTIVE("Automotive"),
    BANKING("Banking"),
    INDUSTRIAL_TECH("Industrial"),
    INSURANCE_FINANCE("Insurance"),
    TELECOMMUNICATIONS("Telecom"),
    IT_SOFTWARE("IT"),
    PHARMA_CHEMICAL("Pharma"),
    OTHER("Other")
}

// Create a new DataFrame by converting the "period" column to LocalDate using the custom format
// and converting "percentageDebtToEquity" column to Double,
// then sorting based on "company" and "period", and finally adding a "sector" column
// depending on the company name
val companiesDf = dataFrame
    .convert { period }.with { LocalDate.parse(it, format) }
    .convert { percentageDebtToEquity }.with { it.removeSuffix("%").replace(',', '.').toDouble() }
    .convert { roa and roe }.with { it.replace(".", "").toDouble() }
    .sortBy { company and period }
    .add("sector") {
        when (company) {
            "Volkswagen AG", "BMW AG", "Daimler AG", "Porsche AG" -> BusinessSector.AUTOMOTIVE
            "Siemens AG", "BASF SE" -> BusinessSector.INDUSTRIAL_TECH
            "Allianz SE" -> BusinessSector.INSURANCE_FINANCE
            "Deutsche Bank AG" -> BusinessSector.BANKING
            "Deutsche Telekom AG" -> BusinessSector.TELECOMMUNICATIONS
            "SAP SE" -> BusinessSector.IT_SOFTWARE
            "Bayer AG", "Merck KGaA" -> BusinessSector.PHARMA_CHEMICAL
            else -> BusinessSector.OTHER
        }
    }

### Aggregating Financial Data

These steps group data by company and calculate key metrics (mean, median, std, min, max) for financial columns like revenue, net income, and ratios.

In [20]:
companiesDf.groupBy { company }.aggregate {
    val financeColumns = it.select { revenue and netIncome and liabilities and assets and equity and roa and roe and debtToEquity and percentageDebtToEquity }
    financeColumns.mean() into "mean"
    financeColumns.median() into "median"
    financeColumns.std() into "std"
    financeColumns.min() into "min"
    financeColumns.max() into "max"
}

<iframe onload="o_resize_iframe_out_6()" style="width:100%;" class="result_container" id="iframe_out_6" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_1241513994"></table>

<p class="dataframe_description">DataFrame: rowsCount = 12, columnsCount = 6</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 

In [21]:
// Group by "company" and aggregate key financial columns
companiesDf.groupBy { sector }.aggregate {
    val financeColumns = it.select { revenue and netIncome and liabilities and assets and equity and roa and roe and debtToEquity and percentageDebtToEquity }
    financeColumns.mean() into "mean"
    financeColumns.median() into "median"
    financeColumns.std() into "std"
    financeColumns.min() into "min"
    financeColumns.max() into "max"
}

sector,mean,,,,,,,,median,,,,,,,,,std,,,,,,,,min,,,,,,,,,max,,,,,,,,
,revenue,netIncome,liabilities,assets,equity,roa,roe,percentageDebtToEquity,revenue,netIncome,liabilities,assets,equity,roa,roe,debtToEquity,percentageDebtToEquity,revenue,netIncome,liabilities,assets,equity,roa,roe,percentageDebtToEquity,revenue,netIncome,liabilities,assets,equity,roa,roe,debtToEquity,percentageDebtToEquity,revenue,netIncome,liabilities,assets,equity,roa,roe,debtToEquity,percentageDebtToEquity
INSURANCE_FINANCE,13015427603.000000,1314983974.006250,29927069416.812500,56971548896.812500,27044479479.843750,2086997152.625000,3437586364.281250,451.397500,13380302290.000000,1240886942.000000,30751494230.000000,56314634789.000000,27995269353.000000,1511668947.500000,3325694133.500000,2.356.716.444,369.450000,4031726568.538354,590182030.528996,11841968487.721144,15082579450.801540,8964325169.429451,1886302018.904862,2043975922.447980,497.898219,5318114486,373428560.200000,10477167725,30329911272,11830459986,174587839.000000,611965731.000000,1.031.419.625,0.000000,19814938566,2402780363.000000,49692687138,93604660446,47947280377,7368255218.000000,7893783146.000000,983.597.322,2293.350000
INDUSTRIAL_TECH,12934714693.750000,1223305504.875000,29649972651.265625,61284478201.109375,31634505549.875000,2090295909.765625,3587689393.734375,455.869062,12274566346.000000,1171189842.000000,29454204370.500000,59053475202.500000,33939244833.000000,1863036066.500000,3394789346.000000,3.948.375.437,373.720000,4085478693.957651,556763437.220554,11716214952.053246,16741825253.907963,11508005629.635681,1494884669.783042,2166034306.212876,547.277337,5703877749,459390813.500000,10137976567,29444881442,10556038645,8183968.000000,45603762.000000,1.005.532.267,0.000000,19846387618,2884007106.000000,49603599758,95650487131,49441756560,9156051924.000000,9827624651.000000,994.554.924,2617.560000
AUTOMOTIVE,12501654488.507812,1247645639.892969,31070048020.976562,61259037225.367190,30188989204.414062,2118081428.039063,3235733994.031250,416.289219,12586633987.000000,1149659115.500000,32950291088.500000,60822980273.500000,30063127704.000000,1878812649.000000,2786218764.000000,240.707.213,341.475000,4046388553.197570,517499635.250430,10919839740.967953,17518169422.712760,12459491566.511446,1540731600.206889,2239985575.500741,441.890333,5152922484,298030924.100000,10690275300,21773242219,10404620558,17386781.000000,124523492.000000,1.000.804.404,0.000000,19909637251,2732548048.000000,49769403556,95156833814,49918115168,8672214379.000000,9878142355.000000,995.319.063,1958.370000
PHARMA_CHEMICAL,12854888417.421875,1319691782.120313,33209900694.328125,61735172337.765625,28525271643.500000,2187787196.546875,3930715306.390625,435.346406,13018516306.500000,1178237040.000000,32993119668.000000,61395602419.000000,28795214254.500000,2012013515.500000,3681807322.000000,2.952.563.918,333.980000,4038390085.582360,590421621.153112,10955103627.573359,16216439910.641758,11108094353.332819,1297937669.004892,2464727270.115643,436.512966,5728072323,389688114.000000,10046371516,27849556029,10017618918,977949.000000,47691775.000000,1.034.227.784,0.000000,19647577142,2691544192.000000,49354599469,95054854077,49110493703,6055903707.000000,9607657241.000000,987.443.008,1818.110000
BANKING,12552273759.250000,1254325661.253125,31804324927.843750,64484873493.968750,32680548566.156250,2042935983.156250,3542614134.843750,398.719688,11868501230.000000,1101588195.500000,33874325159.000000,64508527196.000000,32899597047.000000,1688790297.000000,2806814710.500000,300.121.019,293.870000,4429631424.406467,576058631.887190,12815920189.273525,17260497429.425790,11041972482.317745,1230001154.483563,2106292761.582215,486.555376,5220250485,375695945.900000,10546244209,28137106192,11728609038,107264457.000000,229299295.000000,1.037.275.695,0.000000,19868905162,2752792674.000000,49199711543,92083567435,49507409035,4955368641.000000,8550802927.000000,981.504.776,2008.380000
TELECOMMUNICATIONS,12948836356.781250,1242393628.9

In [22]:
companiesDf.groupBy { sector }.aggregate {
    revenue.mean() into "Avg revenue"
    revenue.sum() into "Total revenue"
    netIncome.mean() into "Avg Net Income"
    netIncome.sum() into "Sum Net Income"
    roa.mean() into "Avg roa"
    roe.mean() into "Avg roe"
}.sortBy { sector }

sector,Avg revenue,Total revenue,Avg Net Income,Sum Net Income,Avg roa,Avg roe
AUTOMOTIVE,12501654488.507812,1600211774529,1247645639.892969,159698641906.300000,2118081428.039063,3235733994.031250
BANKING,12552273759.250000,401672760296,1254325661.253125,40138421160.100000,2042935983.156250,3542614134.843750
INDUSTRIAL_TECH,12934714693.750000,827821740400,1223305504.875000,78291552312.000000,2090295909.765625,3587689393.734375
INSURANCE_FINANCE,13015427603.000000,416493683296,1314983974.006250,42079487168.200005,2086997152.625000,3437586364.281250
TELECOMMUNICATIONS,12948836356.781250,414362763417,1242393628.968750,39756596126.999985,1907479731.343750,3626682563.343750
IT_SOFTWARE,12416678253.562500,397333704114,1100020966.293750,35200670921.400000,1803926622.593750,3279309301.281250
PHARMA_CHEMICAL,12854888417.421875,822712858715,1319691782.120313,84460274055.700000,2187787196.546875,3930715306.390625


In [23]:
// Group by "period" and "sector" then compute total revenue and net income
val timeSerDf = companiesDf.groupBy { period and sector }.aggregate {
    revenue.sum() into "totalRevenue"
    netIncome.sum() into "totalNetIncome"
}

// List of business sectors
val listOfSectors = listOf(
    BusinessSector.AUTOMOTIVE,
    BusinessSector.BANKING,
    BusinessSector.INSURANCE_FINANCE,
    BusinessSector.INDUSTRIAL_TECH,
    BusinessSector.TELECOMMUNICATIONS,
    BusinessSector.IT_SOFTWARE,
    BusinessSector.PHARMA_CHEMICAL
)

// Matching colors for each sector
val listOfSectorColors = listOf(
    Color.hex("#ffaf00"),
    Color.hex("#f46920"),
    Color.hex("#f53255"),
    Color.hex("#f857c1"),
    Color.hex("#29bdfd"),
    Color.hex("#00cbbf"),
    Color.hex("#01c159")
)

## Visualizing Revenue and Net Income by Sector

1. Revenue by Sector:
    - A line chart shows total revenue over time, grouped by business sector.
    - Points highlight specific values, and each sector is color-coded using a predefined palette.
    - The chart includes a legend for sector identification.
2. Net Income by Sector:
    - A similar line chart displays total net income over time for each sector.
    - Points and color-coding are used to enhance clarity, with a legend indicating the sectors.

These visualizations help analyze trends and compare financial performance across sectors over time.

In [24]:
// Plot total revenue by period and sector
timeSerDf.plot {
    // Map the x-axis to the "period" column
    x(period) { axis.name = "Date" }
    // Map the y-axis to the aggregated "totalRevenue"
    y(totalRevenue) { axis.name = "Revenue" }

    // Draw a line chart
    line {
        // Color lines by the "sector" column
        color(sector) {
            // Use a categorical color scale with predefined colors and sectors
            scale = categorical(range = listOfSectorColors, domain = listOfSectors)
            // Configure and label the legend
            legend {
                name = "Sector"
                this.breaksLabeled(
                    BusinessSector.AUTOMOTIVE to BusinessSector.AUTOMOTIVE.simpleName,
                    BusinessSector.BANKING to BusinessSector.BANKING.simpleName,
                    BusinessSector.INSURANCE_FINANCE to BusinessSector.INSURANCE_FINANCE.simpleName,
                    BusinessSector.INDUSTRIAL_TECH to BusinessSector.INDUSTRIAL_TECH.simpleName,
                    BusinessSector.TELECOMMUNICATIONS to BusinessSector.TELECOMMUNICATIONS.simpleName,
                    BusinessSector.IT_SOFTWARE to BusinessSector.IT_SOFTWARE.simpleName,
                    BusinessSector.PHARMA_CHEMICAL to BusinessSector.PHARMA_CHEMICAL.simpleName
                )
            }
        }
    }
    // Add points on top of the line chart
    points {
        size = 3.0
        color(sector) { scale = categorical(range = listOfSectorColors, domain = listOfSectors) }
    }

    // Adjust the layout and overall plot appearance
    layout {
        title = "Revenue by Sector"
        size = 875 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="sxPOQz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Revenue by Sector"
},
"mapping":{
},
"data":{
"period":[1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12,1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12,1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12,1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12,1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12,1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12,1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E12,1.6724448E12,1.6802208E12,1.6880832E12,1.696032E12,1.7039808E12,1.7118432E12,1.7197056E12,1.7276544E12,1.7356032E12],
"totalRevenue":[9.164246774E9,1.4340884258E10,7.168009253E9,1.9458831198E10,1.7610247426E10,1.1661846976E10,1.0545031789E10,1.1358222175E10,1.1744716344E10,9.612688482E9,1.3700664809E10,1.5508664175E10,9.263668849E9,1.3787621203E10,1.800594671E10,1.0098889889E10,8.226780982E9,1.7019949886E10,1.107363031E10,9.38055538E9,1.6640634818E10,8.08976887E9,1.6931011156E10,1.6472180169E10,1.4581130415E10,5.318114486E9,1.9814938566E10,1.3941085692E10,1.8801502624E10,1.3059939771E10,1.6908879786E10,7.203400075E9,3.6669696415E10,2.5204994829E10,1.5251884303E10,3.6611818279E10,2.6378338058E10,2.274690891E10,3.2481361756E10,2.0008293226

In [25]:
// Plot total net income by period and sector
timeSerDf.plot {
    // Map the x-axis to the "period" column
    x(period) { axis.name = "Date" }
    // Map the y-axis to the aggregated "totalNetIncome"
    y(totalNetIncome) { axis.name = "Net Income" }

    // Draw a line chart
    line {
        // Color lines by the "sector" column
        color(sector) {
            // Use the same categorical color scale and sector list
            scale = categorical(range = listOfSectorColors, domain = listOfSectors)
            // Configure and label the legend
            legend {
                name = "Sector"
                this.breaksLabeled(
                    BusinessSector.AUTOMOTIVE to BusinessSector.AUTOMOTIVE.simpleName,
                    BusinessSector.BANKING to BusinessSector.BANKING.simpleName,
                    BusinessSector.INSURANCE_FINANCE to BusinessSector.INSURANCE_FINANCE.simpleName,
                    BusinessSector.INDUSTRIAL_TECH to BusinessSector.INDUSTRIAL_TECH.simpleName,
                    BusinessSector.TELECOMMUNICATIONS to BusinessSector.TELECOMMUNICATIONS.simpleName,
                    BusinessSector.IT_SOFTWARE to BusinessSector.IT_SOFTWARE.simpleName,
                    BusinessSector.PHARMA_CHEMICAL to BusinessSector.PHARMA_CHEMICAL.simpleName
                )
            }
        }

    }

    // Add points on top of the line chart
    points {
        size = 3.0
        color(sector) { scale = categorical(range = listOfSectorColors, domain = listOfSectors) }
    }

    // Adjust the layout and overall plot appearance
    layout {
        title = "Net Income by Sector"
        size = 875 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UnkDQD"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Net Income by Sector"
},
"mapping":{
},
"data":{
"totalNetIncome":[9.810419526E8,1.865680236E9,3.734285602E8,1.6001071E9,2.015329868E9,1.354192659E9,1.010594241E9,1.091045445E9,1.284599881E9,1.071424359E9,8.928518107E8,1.417702229E9,6.356711668E8,1.605393739E9,2.292027552E9,1.161225924E9,5.228581145E8,2.402780363E9,9.128224557E8,1.350100953E9,2.400803163E9,4.880849353E8,2.086994153E9,2.043609431E9,1.529637744E9,4.31100966E8,1.836312727E9,1.722322782E9,1.006058196E9,8.648264405E8,1.197174003E9,6.316840179E8,3.619914311E9,2.0135558614E9,1.8256687004E9,3.073921918E9,2.592402013E9,2.2431743535E9,3.625281841E9,1.8374034841E9,2.587129058E9,2.2476121194E9,1.873853311E9,2.636605236E9,2.032992803E9,1.581811868E9,3.3990455275E9,3.301410261E9,2.7620134115E9,2.2850664771E9,2.7320701631E9,3.6261046096E9,9.892885743E8,3.813293766E9,1.2191344964E9,2.6226312015E9,1.7869190812E9,2.920715173E9,1.7351830173E9,1.1435008994E9,2.538658145E9,2.1077745777E9,3.072908584E9,2.4445074686E9,3.8320068799E9,3.5158679689E9,6.0266370031E9,4.1142500552999997E9,4.3531814776E9,5.9351835274E9,4.827665167E9,4.5814406473E9,5.234245716E9,4.5906669647E9,4.0370077918E9,4.8957466549E9,5.327778615E9,5.9502908207E9,4.7797836119E9,3.4387207696E9,4.591492407E9,4.6713649123E9,5.85932151E9,7.042810945E9,5.5800318282E9,4.979117442E9,5.012355407E9,2.6838512436E9,4.4921163747E9,4.8636776247E9,6.485926571E9,4.0313459267E9,5.5323139883E9,5.6711413787E9,6.9536539285E9,5.8076467475E9,1.51703145E9,1.9051801255E9,2.598920794E9,3.482877063E9,1.5258286471999998E9,2.1621804412E9,2.1423110603E9,3.40745135E9,3.190593511E9,3.331905921E9,2.2206578769E9,2.935019137E9,2.262173998E9,2.7334968468E9,1.7066238324E9,1.7120495462E9,3.493295895E9,2.827797856E9,2.8975084415E9,3.159899207E9,1.3239766236E9,2.7320485685E9,1.8528008702E9,2.3628552281E9,4.328735006E9,2.1873731219E9,2.276420415E9,3.009574937E9,4.204408213E9,3.500082181E9,2.4507651521E9,3.0184307393E9,1.680873545E9,2.752792674E9,1.429630591E9,9.585083337E8,1.900766775E9,6.63079894E8,1.082288023E9,1.274941171E9,1.56307922E9,7.665202339E8,3.756959459E8,5.764513525E8,9.802851594E8,1.326310547E9,1.309520126E9,1.761523797E9,1.028066418E9,4.604580021E8,7.828401803E8,1.120888368E9,1.232326295E9,1.369609294E9,1.074208571E9,9.495517012E8,8.878138823E8,8.041823594E8,1.449472284E9,9.670970575E8,2.333015849E9,7.298478769E8,2.32302217E9,2.223753463E9,1.348710564E9,9.243459058E8,1.163491999E9,1.425682028E9,1.967928248E9,1.025524147E9,9.862802506E8,1.698666925E9,1.112803764E9,1.035650167E9,1.380378045E9,8.050162658E8,1.501884164E9,2.209253956E9,1.310559971E9,1.231038198E9,8.319217073E8,3.789515381E8,1.480956904E9,2.116759905E9,4.406495401E8,1.187007039E9,1.613587509E9,9.904204744E8,1.439735669E9,5.617320015E8,7.506132751E8,8.841674466E8,1.007117429E9,1.23897666E9,8.727675317E8,2.834016899E9,7.190909712E8,7.448730037E8,1.997618536E9,2.276360916E9,1.421900578E9,5.418641959E8,7.506011032E8,1.365996842E9,9.899343613E8,9.322714785E8,1.079900966E9,6.621652307E8,5.299828773E8,9.411958262E8,2.071404633E9,1.027662516E9,7.312162981E8,7.199498675E8,1.079598533E9,8.009833004E8,1.471513321E9,1.79214363E9,1.701020228E9,1.837934274E9,8.630460669E8,6.987886225E8,1.338393789E9,1.069408471E9,7.613065162E8,5.292573654E8,8.302799962E8,9.230066072E8],
"period":[1.4909184E12,1.4987808E12,1.5067296E12,1.5146784E12,1.5224544E12,1.5303168E12,1.5382656E12,1.5462144E12,1.5539904E12,1.5618528E12,1.5698016E12,1.5777504E12,1.5856128E12,1.5934752E12,1.601424E12,1.6093728E12,1.6171488E12,1.6250112E12,1.63296E12,1.6409088E12,1.6486848E12,1.6565472E12,1.664496E

## roa and roe Analysis by Sector

1. Computing Averages and Standard Deviations:
    - Group the data by sector and calculate the mean and standard deviation for Return on Assets (roa) and Return on Equity (roe).
    - This creates a summarized dataset for sector-level performance comparison.
2. Visualizing roa by Sector:
    - A bar chart displays the average roa for each sector.
    - Error bars represent one standard deviation, showing the variability within each sector.
3. Visualizing roe by Sector:
    - A similar bar chart illustrates the average roe across sectors.
    - Error bars provide insight into the standard deviation of roe within each sector.

These charts help compare sector-level profitability metrics and assess consistency within sectors.

In [26]:
// Group data by sector to compute average and standard deviations of roa and roe
val roeAndRoaDf = companiesDf.groupBy { sector }.aggregate {
    roa.mean() into "Avg roa"
    roa.std() into "Std roa"
    roe.mean() into "Avg roe"
    roe.std() into "Std roe"
}

roeAndRoaDf

sector,Avg roa,Std roa,Avg roe,Std roe
INSURANCE_FINANCE,2086997152.625000,1886302018.904862,3437586364.281250,2043975922.447980
INDUSTRIAL_TECH,2090295909.765625,1494884669.783042,3587689393.734375,2166034306.212876
AUTOMOTIVE,2118081428.039063,1540731600.206889,3235733994.031250,2239985575.500741
PHARMA_CHEMICAL,2187787196.546875,1297937669.004892,3930715306.390625,2464727270.115643
BANKING,2042935983.156250,1230001154.483563,3542614134.843750,2106292761.582215
TELECOMMUNICATIONS,1907479731.343750,1262589468.122650,3626682563.343750,2258582990.930430
IT_SOFTWARE,1803926622.593750,1235422951.830178,3279309301.281250,2326900921.602219


In [27]:
// Plot average roa by sector with error bars representing one standard deviation
roeAndRoaDf.plot {
    // Set the x-axis to the sector names
    x(sector.map { it.simpleName }) { axis.name = "Sector of Business" }

    bars {
        // Use the "Avg roa" column for the bar heights
        y(`Avg roa`) { scale = continuous(min = .0, max = 4.5e+9) }
        // Fill bars with a chosen color
        fillColor = Color.hex("#ffaf00")
    }
    lineRanges {
        // Calculate the min and max for the error bars (Std roa)
        yMin(`Avg roa`.toList().zip(`Std roa`.toList()).map { it.first - it.second })
        yMax(`Avg roa`.toList().zip(`Std roa`.toList()).map { it.first + it.second })
        // Color the line of the ranges
        borderLine.color = Color.GREY
    }

    // Adjust layout options such as title and overall size
    layout {
        title = "Average roa By Sector With Standard Deviation"
        size = 875 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="6M8PVd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Average roa By Sector With Standard Deviation"
},
"mapping":{
},
"data":{
"ymin":[2.0069513372013807E8,5.95411239982583E8,5.773498278321731E8,8.898495275419831E8,8.12934828672687E8,6.448902632210996E8,5.68503670763572E8],
"ymax":[3.973299171529862E9,3.585180579548667E9,3.6588130282459517E9,3.485724865551767E9,3.272937137639813E9,3.1700691994664E9,3.0393495744239283E9],
"Avg roa":[2.086997152625E9,2.090295909765625E9,2.1180814280390625E9,2.187787196546875E9,2.04293598315625E9,1.90747973134375E9,1.80392662259375E9],
"sector":["Insurance","Industrial","Automotive","Pharma","Banking","Telecom","IT"]
},
"ggsize":{
"width":875.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Sector of Business"
},{
"aesthetic":"y",
"limits":[0.0,4.5E9]
},{
"aesthetic":"x",
"discrete":true,
"name":"Sector of Business"
}],
"layers":[{
"mapping":{
"x":"sector",
"y":"Avg roa"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#ffaf00",
"data":{
}
},{
"mapping":{
"x":"sector",
"ymin":"ymin",
"ymax":"ymax"
},
"stat":"identity",
"color":"#a39999",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"linerange",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"sector"
},{
"type":"float",
"column":"Avg roa"
},{
"type":"float",
"column":"ymin"
},{
"type":"float",
"column":"ymax"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("6M8PVd");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 875.0,
 height: 500.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Insurance 
 
 
 
 
 
 
 
 
 Industrial 
 
 
 
 
 
 
 
 
 Automotive 
 
 
 
 
 
 
 
 
 Pharma 
 
 
 
 
 
 
 
 
 Banking 
 
 
 
 
 
 
 
 
 Telecom 
 
 
 
 
 
 
 
 
 IT 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500M 
 
 
 
 
 
 
 1G 
 
 
 
 
 
 
 2G 
 
 
 
 
 
 
 2G 
 
 
 
 
 
 
 3G 
 
 
 
 
 
 
 3G 
 
 
 
 
 
 
 4G 
 
 
 
 
 
 
 4G 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 
 
 Average roa By Sector With Standard Deviation 
 
 
 
 
 Avg roa 
 
 
 
 
 Sector of Business

In [28]:
// Plot average roe by sector with error bars representing one standard deviation
roeAndRoaDf.plot {
    // Set the x-axis to the sector names
    x(sector.map { it.simpleName }) { axis.name = "Sector of Business" }

    bars {
        // Use the "Avg roe" column for the bar heights
        y(`Avg roe`)
        // Fill bars with a chosen color
        fillColor = Color.hex("#ffaf00")
    }
    lineRanges {
        // Calculate the min and max for the error bars (Std roe)
        yMin(`Avg roe`.toList().zip(`Std roe`.toList()).map { it.first - it.second })
        yMax(`Avg roe`.toList().zip(`Std roe`.toList()).map { it.first + it.second })
        // Color the line of the ranges
        borderLine.color = Color.GREY
    }

    // Adjust layout options such as title and overall size
    layout {
        title = "Average roe By Sector With Standard Deviation"
        size = 875 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bVaxUr"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Average roe By Sector With Standard Deviation"
},
"mapping":{
},
"data":{
"ymin":[1.3936104418332696E9,1.4216550875214992E9,9.957484185305095E8,1.4659880362749825E9,1.4363213732615347E9,1.36809957241332E9,9.524083796790309E8],
"ymax":[5.481562286729231E9,5.75372369994725E9,5.47571956953199E9,6.395442576506268E9,5.648906896425965E9,5.885265554274179E9,5.606210222883469E9],
"Avg roe":[3.43758636428125E9,3.587689393734375E9,3.23573399403125E9,3.930715306390625E9,3.54261413484375E9,3.62668256334375E9,3.27930930128125E9],
"sector":["Insurance","Industrial","Automotive","Pharma","Banking","Telecom","IT"]
},
"ggsize":{
"width":875.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Sector of Business"
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":"Sector of Business"
}],
"layers":[{
"mapping":{
"x":"sector",
"y":"Avg roe"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#ffaf00",
"data":{
}
},{
"mapping":{
"x":"sector",
"ymin":"ymin",
"ymax":"ymax"
},
"stat":"identity",
"color":"#a39999",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"linerange",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"sector"
},{
"type":"float",
"column":"Avg roe"
},{
"type":"float",
"column":"ymin"
},{
"type":"float",
"column":"ymax"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("bVaxUr");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 875.0,
 height: 500.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Insurance 
 
 
 
 
 
 
 
 
 Industrial 
 
 
 
 
 
 
 
 
 Automotive 
 
 
 
 
 
 
 
 
 Pharma 
 
 
 
 
 
 
 
 
 Banking 
 
 
 
 
 
 
 
 
 Telecom 
 
 
 
 
 
 
 
 
 IT 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1G 
 
 
 
 
 
 
 2G 
 
 
 
 
 
 
 3G 
 
 
 
 
 
 
 4G 
 
 
 
 
 
 
 5G 
 
 
 
 
 
 
 6G 
 
 
 
 
 
 
 
 
 Average roe By Sector With Standard Deviation 
 
 
 
 
 Avg roe 
 
 
 
 
 Sector of Business